# 뉴럴넷으로 패션 아이템 구분하기
Fashion MNIST 데이터셋과 앞서 배운 인공신경망을 이용하여 패션아이템을 구분해봅니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
print(USE_CUDA, DEVICE)

False cpu


In [5]:
EPOCHS = 30
BATCH_SIZE = 64

## 데이터셋 불러오기

In [6]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [7]:
trainset = datasets.FashionMNIST(
    root      = './.data/', 
    train     = True,
    download  = True,
    transform = transform
)
testset = datasets.FashionMNIST(
    root      = './.data/', 
    train     = False,
    download  = True,
    transform = transform
)

train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True,
)
test_loader = torch.utils.data.DataLoader(
    dataset     = testset,
    batch_size  = BATCH_SIZE,
    shuffle     = True,
)

## 뉴럴넷으로 Fashion MNIST 학습하기

입력 `x` 는 `[배치크기, 색, 높이, 넓이]`로 이루어져 있습니다.
`x.size()`를 해보면 `[64, 1, 28, 28]`이라고 표시되는 것을 보실 수 있습니다.
Fashion MNIST에서 이미지의 크기는 28 x 28, 색은 흑백으로 1 가지 입니다.
그러므로 입력 x의 총 특성값 갯수는 28 x 28 x 1, 즉 784개 입니다.

우리가 사용할 모델은 3개의 레이어를 가진 인공신경망 입니다. 

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 모델 준비하기

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다.
일반적으로 CPU 1개만 사용할 경우 필요는 없지만,
GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다.
지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [9]:
model        = Net().to(DEVICE)
optimizer    = optim.SGD(model.parameters(), lr=0.01)

## 학습하기

In [10]:
def train(model, train_loader, optimizer):
    ### train mode 설정
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #### 학습 데이터를 DEVICE의 메모리로 보냄
        data, target = data.to(DEVICE), target.to(DEVICE)
        ### 기울기 값 zero
        optimizer.zero_grad()
        output = model(data)
        
        ### 오차 함수 creoo etnropy 사용
        loss = F.cross_entropy(output, target)
        
        ### backpropagation 
        loss.backward()
        optimizer.step()

## 테스트하기

In [13]:
def evaluate(model, test_loader):
    ### evaluation mode 설정
    model.eval()
    test_loss = 0
    correct = 0
    
    ### 평가하는 과정에서는 기울기를 계산하지 않아도 됩니다. with torch.no_grad() 추가
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 모든 오차 더하기
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더합니다.
            # output.max() 함수는 가장 큰값과 그 값의 인덱스 추출
            pred = output.max(1, keepdim=True)[1]
            # eq () : 함수는 값이 일치하면 1, 아니면 0 
            # view_as : Target텐서를 view_as ()함수 안에 들어가는 인수
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [14]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 0.3967, Accuracy: 85.72%
[2] Test Loss: 0.3786, Accuracy: 86.66%
[3] Test Loss: 0.4415, Accuracy: 84.91%
[4] Test Loss: 0.3578, Accuracy: 87.28%
[5] Test Loss: 0.3616, Accuracy: 86.99%
[6] Test Loss: 0.3678, Accuracy: 86.85%
[7] Test Loss: 0.3730, Accuracy: 86.66%
[8] Test Loss: 0.3592, Accuracy: 87.17%
[9] Test Loss: 0.3588, Accuracy: 86.99%
[10] Test Loss: 0.3430, Accuracy: 87.62%
[11] Test Loss: 0.3667, Accuracy: 86.86%
[12] Test Loss: 0.3575, Accuracy: 86.98%
[13] Test Loss: 0.3598, Accuracy: 87.08%
[14] Test Loss: 0.3409, Accuracy: 87.97%
[15] Test Loss: 0.3420, Accuracy: 87.55%
[16] Test Loss: 0.3621, Accuracy: 87.06%
[17] Test Loss: 0.3491, Accuracy: 87.71%
[18] Test Loss: 0.3480, Accuracy: 87.37%
[19] Test Loss: 0.3342, Accuracy: 88.17%
[20] Test Loss: 0.3499, Accuracy: 87.32%
[21] Test Loss: 0.3398, Accuracy: 87.72%
[22] Test Loss: 0.3301, Accuracy: 88.35%
[23] Test Loss: 0.3617, Accuracy: 86.94%
[24] Test Loss: 0.3519, Accuracy: 87.52%
[25] Test Loss: 0.3436, A